In [2]:
import sys
import torch
from torchvision import transforms

sys.path.append('..')

from src.models.models import *
from src.models.aligner import *

/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sample_image = torch.rand(1, 3, 224, 224)
sample_test = ["Wlazl kotek na plotek i mruga"]
batch_sample_image = torch.rand(2, 3, 224, 224)
batch_sample_test = ["Wlazl kotek na plotek i mruga", "Wlazl kotek na plotek i mruga na ciebie"]

In [4]:
dino_family = DinoVisionModel
dino_family.SHAPES

{'dinov2_vits14': (384, 256),
 'dinov2_vitb14': (768, 256),
 'dinov2_vitl14': (1024, 256),
 'dinov2_vitg14': (1536, 256)}

In [5]:
base_dino = dino_family(checkpoint="dinov2_vitb14", pooling="cls")
base_dino

2023-11-23 20:59:03.821 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
xFormers not available
xFormers not available


DinoVisionModel(
  (_model): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        

In [6]:
trans = transforms.Compose(base_dino.get_basic_transformations())
trans(sample_image).shape, base_dino(sample_image).shape

/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(torch.Size([2, 3, 224, 224]), torch.Size([2, 768]))

In [7]:
with torch.inference_mode():
    output_image = base_dino(trans(sample_image))
    output_batch_image = base_dino(trans(batch_sample_image))

In [8]:
output_image.shape, output_batch_image.shape, base_dino.output_dim

(torch.Size([2, 768]), torch.Size([2, 768]), 768)

In [9]:
with torch.inference_mode():
    for pooler in ["cls", "mean", "max", "mean_max", "cls_mean", "cls_max", "cls_mean_max"]:
        for checkpoint in ["dinov2_vits14", "dinov2_vitb14"]:#, "dinov2_vitl14", "dinov2_vitg14"]:
            dino = dino_family(checkpoint=checkpoint, pooling=pooler)
            print(f"pooler: {pooler}, checkpoint: {checkpoint}, output_dim: {dino.output_dim}, output_shape: {dino(sample_image).shape}, batch_output_shape: {dino(batch_sample_image).shape}")

2023-11-23 20:59:09.159 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-23 20:59:10.865 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: cls, checkpoint: dinov2_vits14, output_dim: 384, output_shape: torch.Size([2, 384]), batch_output_shape: torch.Size([2, 384])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-23 20:59:14.711 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: cls, checkpoint: dinov2_vitb14, output_dim: 768, output_shape: torch.Size([2, 768]), batch_output_shape: torch.Size([2, 768])


...

pooler: cls_max, checkpoint: dinov2_vitb14, output_dim: 1024, output_shape: torch.Size([2, 1024]), batch_output_shape: torch.Size([2, 1024])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-23 20:59:38.756 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: cls_mean_max, checkpoint: dinov2_vits14, output_dim: 896, output_shape: torch.Size([2, 896]), batch_output_shape: torch.Size([2, 896])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main


pooler: cls_mean_max, checkpoint: dinov2_vitb14, output_dim: 1280, output_shape: torch.Size([2, 1280]), batch_output_shape: torch.Size([2, 1280])


In [10]:
try:
    dino_family(checkpoint="dinov2_vits14", pooling="Wrong")
    assert False
except ValueError:
    assert True

2023-11-23 20:59:41.814 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main


ValueError: Pooling Wrong is not supported for DinoVisionModel.

In [3]:
robert_family = RobertaCaptionModel
robert_family.SHAPES

{'roberta-base': 768, 'roberta-large': 1024, 'xlm-roberta-base': 250002}

In [4]:
base_robert = robert_family(checkpoint="roberta-base", pooling="cls")
base_robert

2023-11-23 21:41:20.784 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 21:41:23.326 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.


RobertaCaptionModel(
  (_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [5]:
tokenizer = base_robert.get_tokenizer()
tokenizer(sample_test, return_tensors="pt", padding=True).keys(), tokenizer(batch_sample_test, return_tensors="pt", padding=True).keys()

(dict_keys(['input_ids', 'attention_mask']),
 dict_keys(['input_ids', 'attention_mask']))

In [6]:
with torch.inference_mode():
    output_test = base_robert(tokenizer(sample_test, return_tensors="pt", padding=True))
    output_batch_test = base_robert(tokenizer(batch_sample_test, return_tensors="pt", padding=True))

In [7]:
output_test.shape, output_batch_test.shape, base_robert.output_dim

(torch.Size([1, 768]), torch.Size([2, 768]), 768)

In [8]:
for pooler in ["cls", "mean", "max", "mean_max", "cls_mean", "cls_max", "cls_mean_max"]:
    for checkpoint in ["roberta-base", "xlm-roberta-base"]:
        robert = robert_family(checkpoint=checkpoint, pooling=pooler)
        print(f"pooler: {pooler}, checkpoint: {checkpoint}, output_dim: {robert.output_dim}, output_shape: {robert(tokenizer(sample_test, return_tensors='pt', padding=True)).shape}, batch_output_shape: {robert(tokenizer(batch_sample_test, return_tensors='pt', padding=True)).shape}")

2023-11-23 21:41:23.461 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 21:41:25.484 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 21:41:25.579 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


pooler: cls, checkpoint: roberta-base, output_dim: 768, output_shape: torch.Size([1, 768]), batch_output_shape: torch.Size([2, 768])


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-11-23 21:41:31.254 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 21:41:31.478 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


pooler: cls, checkpoint: xlm-roberta-base, output_dim: 250002, output_shape: torch.Size([1, 250002]), batch_output_shape: torch.Size([2, 250002])


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 21:41:32.907 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 21:41:33.394 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


...

pooler: cls_max, checkpoint: xlm-roberta-base, output_dim: 500004, output_shape: torch.Size([1, 500004]), batch_output_shape: torch.Size([2, 500004])


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 21:42:12.249 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 21:42:12.824 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.


pooler: cls_mean_max, checkpoint: roberta-base, output_dim: 2304, output_shape: torch.Size([1, 2304]), batch_output_shape: torch.Size([2, 2304])


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-11-23 21:42:18.266 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.


pooler: cls_mean_max, checkpoint: xlm-roberta-base, output_dim: 750006, output_shape: torch.Size([1, 750006]), batch_output_shape: torch.Size([2, 750006])


In [9]:
try:
    robert_family(checkpoint="roberta-base", pooling="Wrong")
    assert False
except ValueError:
    assert True

2023-11-23 21:42:18.532 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 21:42:20.184 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.


In [3]:
model = ClipAligner(
    DinoVisionModel(checkpoint="dinov2_vitb14", pooling="cls"),
    RobertaCaptionModel(checkpoint="roberta-base", pooling="cls"),
    vision_layer=[(768, "mlp"), (512, "mlp")],
    caption_layer=[(768, "mlp"), (512, "mlp")],
)

2023-11-23 22:26:24.568 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
xFormers not available
xFormers not available
2023-11-23 22:26:27.402 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 22:26:29.774 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 22:26:29.777 | DEBUG    | src.models.base:__init__:52 - Model image-caption ClipAligner initialized.


In [4]:
trans, tokenizer = model.get_basic_transformations()
trans = transforms.Compose(trans)

In [5]:
with torch.inference_mode():
    output_test = model(trans(sample_image), tokenizer(sample_test, return_tensors="pt", padding=True))
    output_batch_test = model(trans(batch_sample_image), tokenizer(batch_sample_test, return_tensors="pt", padding=True))

/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [6]:
output_test[0].shape, output_batch_test[0].shape, output_test[1].shape, output_batch_test[1].shape, model.output_dim

(torch.Size([1, 512]),
 torch.Size([2, 512]),
 torch.Size([1, 512]),
 torch.Size([2, 512]),
 512)

In [12]:
with torch.inference_mode():
    for vision_layer, caption_layer in [
        ([(768, "mlp"), (512, "mlp")], [(768, "mlp"), (512, "mlp")]),
        ([(768, "conv"), (512, "conv")], [(768, "conv"), (512, "conv")]),
        ([(768, "res"), (512, "res")], [(768, "res"), (512, "res")]),
        ([(768, "res"), (512, "conv"), (256, "mlp")], [(768, "res"), (512, "conv"), (256, "mlp")]),
        ([(768, "conv"), (512, "mlp")], [(768, "res"), (512, "mlp")]),
    ]:
        for pooler in ["cls", "mean", "mean_max", "cls_mean_max"]:
            model = ClipAligner(
                DinoVisionModel(checkpoint="dinov2_vitb14", pooling=pooler),
                RobertaCaptionModel(checkpoint="roberta-base", pooling="cls"),
                vision_layer=vision_layer,
                caption_layer=caption_layer,
            )
            output = model(trans(sample_image), tokenizer(sample_test, return_tensors="pt", padding=True))
            print(f"pooler: {pooler}, vision_layer: {vision_layer}, caption_layer: {caption_layer}, output_dim: {model.output_dim}, output_shape_image: {output[0].shape}, output_shape_caption: {output[1].shape}")

2023-11-23 22:29:05.693 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-23 22:29:08.724 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 22:29:11.300 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 22:29:11.301 | DEBUG    | src.models.base:__init__:52 - Model image-caption ClipAligner initialized.
/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc

pooler: cls, vision_layer: [(768, 'mlp'), (512, 'mlp')], caption_layer: [(768, 'mlp'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-23 22:29:13.950 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 22:29:15.471 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 22:29:15.472 | DEBUG    | src.models.base:__init__:52 - Model image-caption ClipAligner initialized.
2023-11-23 22:29:15.796 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: mean, vision_layer: [(768, 'mlp'), (512, 'mlp')], caption_layer: [(768, 'mlp'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


...

pooler: mean, vision_layer: [(768, 'conv'), (512, 'mlp')], caption_layer: [(768, 'res'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-23 22:30:39.488 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 22:30:41.218 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 22:30:41.219 | DEBUG    | src.models.base:__init__:52 - Model image-caption ClipAligner initialized.
2023-11-23 22:30:41.593 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.


pooler: mean_max, vision_layer: [(768, 'conv'), (512, 'mlp')], caption_layer: [(768, 'res'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-23 22:30:44.237 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 22:30:45.762 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 22:30:45.763 | DEBUG    | src.models.base:__init__:52 - Model image-caption ClipAligner initialized.


pooler: cls_mean_max, vision_layer: [(768, 'conv'), (512, 'mlp')], caption_layer: [(768, 'res'), (512, 'mlp')], output_dim: 512, output_shape_image: torch.Size([1, 512]), output_shape_caption: torch.Size([1, 512])


In [13]:

for layer_vision, layer_caption in [["mlp", "Wrong"], ["Wrong", "mlp"]]:
    try:
        model = ClipAligner(
            DinoVisionModel(checkpoint="dinov2_vitb14", pooling="cls"),
            RobertaCaptionModel(checkpoint="roberta-base", pooling="cls"),
            vision_layer=[(768, layer_vision), (512, layer_vision)],
            caption_layer=[(768, layer_caption), (512, layer_caption)],
        )
        assert False
    except ValueError:
        assert True

2023-11-23 22:38:12.364 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
2023-11-23 22:38:16.588 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 22:38:19.446 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 22:38:19.448 | DEBUG    | src.models.base:__init__:52 - Model image-caption ClipAligner initialized.
2023-11-23 22:38:19.588 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigra

In [4]:
from src.models.factory import create_model

In [5]:
model = create_model(
    {
        "image": {"name": "dinov2", "checkpoint": "dinov2_vitb14", "pooling": "cls"},
        "caption": {"name": "roberta", "checkpoint": "roberta-base", "pooling": "cls"},
        "alignment": {"name": "clip_aligner","vision_layer": [(768, "mlp"), (512, "mlp")], "caption_layer": [(768, "mlp"), (512, "mlp")]},
    }
)

2023-11-23 22:59:06.722 | INFO     | src.models.factory:create_model:50 - Initializing Vision model: dinov2 and Caption model: roberta
2023-11-23 22:59:06.724 | DEBUG    | src.models.base:__init__:16 - Model image DinoVisionModel initialized.
Using cache found in /Users/vladimirzaigrajew/.cache/torch/hub/facebookresearch_dinov2_main
xFormers not available
xFormers not available
2023-11-23 22:59:09.590 | DEBUG    | src.models.base:__init__:32 - Model caption RobertaCaptionModel initialized.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-11-23 22:59:12.442 | INFO     | src.models.models:__init__:84 - Sssssh don't listen to the warning above, Robert can be grumpy.
2023-11-23 22:59:12.443 | DEBUG    | src.models.base:__init__:50 - Model image-ca

In [6]:
trans, tokenizer = model.get_basic_transformations()
trans = transforms.Compose(trans)

In [7]:
with torch.inference_mode():
    output_test = model(trans(sample_image), tokenizer(sample_test, return_tensors="pt", padding=True))
    output_batch_test = model(trans(batch_sample_image), tokenizer(batch_sample_test, return_tensors="pt", padding=True))

/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [8]:
output_test[0].shape, output_batch_test[0].shape, output_test[1].shape, output_batch_test[1].shape, model.output_dim

(torch.Size([1, 512]),
 torch.Size([2, 512]),
 torch.Size([1, 512]),
 torch.Size([2, 512]),
 512)